# Extract pdf data


In [1]:
from util import extractPdfToFile

In [2]:
# extractPdfToFile("https://doi.org/10.48550/arXiv.2211.06956")

extracting pdf data...
pdf extraction finished!


# Bedrock


In [1]:
import boto3
import json

In [43]:
GPT_3 = "gpt-3.5-turbo"
GPT_4 = "gpt-4-0125-preview"
CLAUDE2 = "anthropic.claude-v2"
CLAUDE1 = "anthropic.claude-instant-v1"
CLAUDE21 = "anthropic.claude-v2:1"
TITAN_LITE = "amazon.titan-text-lite-v1"
TITAN_EXPRESS = "amazon.titan-text-express-v1"

In [49]:
from langchain.llms.bedrock import Bedrock

# modelId = "amazon.titan-tg1-large"
# modelId = "anthropic.claude-v2"
# modelId = "meta.llama2-70b-chat-v1"
modelId = CLAUDE2

bedrock = boto3.client(service_name="bedrock-runtime")

llm = Bedrock(
    model_id=modelId,
    model_kwargs={
        # "max_tokens_to_sample": 10,
        # "stopSequences": [],
        "temperature": 0,
        # "top_k": 250
        # "top_p": 1,
    },
    client=bedrock,
)
with open("./prompt.txt", "r") as file:
    prompt_data = file.read()

token_num = llm.get_num_tokens(prompt_data)
print(f"{token_num} tokens")

9458 tokens


In [50]:
llm.invoke(prompt_data)

' Here is a summary of the key points from the text:\n\n- The paper proposes a method called NEUROIMAGEN to reconstruct visual images from EEG brain signals. \n\n- It extracts multi-level semantic information from EEG signals, including pixel-level saliency maps and sample-level textual descriptions.\n\n- The pixel-level saliency map captures fine-grained details like color, position, and shape using a GAN framework. \n\n- The sample-level semantics provide a coarse overview like the image category or caption.\n\n- These multi-level semantics are integrated into a pretrained latent diffusion model to reconstruct high-quality images.\n\n- Experiments on an EEG-image dataset show NEUROIMAGEN can reconstruct images that are perceptually and semantically similar to the actual visual stimuli. \n\n- It outperforms previous methods like Brain2Image in metrics like inception score and semantic accuracy.\n\n- The multi-level semantics at different granularities are shown to be effective for han

In [ ]:
payload = {
    "prompt": "[INST]" + prompt_data + "[/INST]",
    "max_gen_len": 512,
    "temperature": 0.5,
    "top_p": 0.9,
}
body = json.dumps(payload)

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [24]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

Now we have 1 documents and the first one has 6 tokens


In [36]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain

# summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)
summary_chain = load_summarize_chain(llm=llm, chain_type="stuff", verbose=False)

In [37]:
# output = summary_chain.invoke(docs)
output = summary_chain.invoke(prompt_data)
output

AttributeError: 'str' object has no attribute 'page_content'

# vector_db API


In [8]:
import sys

sys.path.append("../vector_db/qdrant/lib")
from llm import LLM

a = LLM("./prompt.txt", "gpt3")
a.get_response()

Hello! I am an AI language model developed by OpenAI. I am designed to assist with various tasks such as answering questions, providing information, and engaging in conversations. I can help with a wide range of topics, so feel free to ask me anything you'd like to know!


IndexError: list index out of range

# Reference


- https://github.com/MemoryLabDev/vector_db/blob/automate-network/qdrant/lib/llm.py

- https://github.com/krishnaik06/AWS-Bedrock
- https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_Summarization/02.long-text-summarization-titan.ipynb

- https://blog.economize.cloud/aws-bedrock-foundation-models-list/ (pricing)
